In [1]:
!pip install transformers
!pip install torch
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 4.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
df = pd.read_csv("/content/Test_file.txt", engine='python',encoding='utf-8', error_bad_lines=False)
df.columns = ["cat", "first", "second", "third","fourth"]
df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,cat,first,second,third,fourth
0,city,اهواز,خوزستان,قشم,هرمزگان
1,city,شیراز,فارس,دامغان,سمنان
2,city,رشت,گیلان,اسلام‌شهر,تهران
3,city,بندرعباس,هرمزگان,ملارد,تهران
4,city,اراک,مرکزی,مریوان,کردستان


In [4]:
tokenizer_ParsBERT = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model_ParsBERT = BertForMaskedLM.from_pretrained('bert-base-multilingual-uncased')

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def predict_masked_word(sentence, model="ParsBERT",relationship=0):
    """ Predicts the masked word in a given sentence using a transformer model and it's tokenizer. """
    # Model Selection
    if(model=="ParsBERT"):
      model = model_ParsBERT
      tokenizer = tokenizer_ParsBERT
    if(model=="Trained ParsBERT"):
      model = trained_bert
      tokenizer = tokenizer_trained_bert  
    # Tokenize the input sentence  
    tokenized_sentence = tokenizer.tokenize("[CLS] " + sentence + "[SEP]")
    mask_index = tokenized_sentence.index("[MASK]")
    token_ids = tokenizer.convert_tokens_to_ids(tokenized_sentence)
    token_tensor = torch.tensor([token_ids])

    # Forward pass without training and backpropagation
    with torch.no_grad():
        outputs = model(token_tensor)
        prediction_logits = outputs[0]
    
    # Calculating 3 nearest prediction of masked token 
    prediction_probs = torch.nn.functional.softmax(prediction_logits[0, mask_index], dim=-1)
    top_k_probs, top_k_token_ids = torch.topk(prediction_probs, 3, sorted=True)

    # Print the predicted tokens for mask and their weight
    predicted_mask = []
    for i in range(3):
        predicted_mask.append(tokenizer.convert_ids_to_tokens([top_k_token_ids[i]])[0])
        token_prob = top_k_probs[i].item()
        print(f"[MASK]: '{predicted_mask[i]}'  : {token_prob}")       
    return predicted_mask[0]   

In [ ]:
df["cat"].unique().count()

array(['city', 'capital', 'currency', 'family', 'gram_thirdperson',
       'gram_past', 'gram_firstperson', 'gram_adj2adv', 'gram_noun2adv',
       'gram_antonym', 'gram_comparative', 'gram_nationality',
       'gram_plural'], dtype=object)

In [6]:
Accuracy = []
Accuracy_city = 0
y= df[df["cat"]=="city"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "city"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "شهر "+df["first"][i[0]] + " در استان " + df["second"][i[0]] + "  در نتیجه شهر " + df["third"][i[0]] + " در استان " + "[MASK]" + " می باشد."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_city = Accuracy_city + 1   
num = y["cat"].count()      
print(Accuracy_city/num)      



اهواز خوزستان قشم هرمزگان
شهر اهواز در استان خوزستان  در نتیجه شهر قشم در استان [MASK] می باشد.
[MASK]: 'فارس'  : 0.5462697744369507
[MASK]: 'کرمان'  : 0.13909097015857697
[MASK]: 'زنجان'  : 0.08805785328149796
شیراز فارس دامغان سمنان
شهر شیراز در استان فارس  در نتیجه شهر دامغان در استان [MASK] می باشد.
[MASK]: 'فارس'  : 0.5272728204727173
[MASK]: 'خراسان'  : 0.16612592339515686
[MASK]: 'اصفهان'  : 0.06654070317745209
رشت گیلان اسلام‌شهر تهران
شهر رشت در استان گیلان  در نتیجه شهر اسلام‌شهر در استان [MASK] می باشد.
[MASK]: 'کرمان'  : 0.32140979170799255
[MASK]: 'فارس'  : 0.31214019656181335
[MASK]: 'زنجان'  : 0.10365597903728485
بندرعباس هرمزگان ملارد تهران
شهر بندرعباس در استان هرمزگان  در نتیجه شهر ملارد در استان [MASK] می باشد.
[MASK]: 'فارس'  : 0.5284292101860046
[MASK]: 'کرمان'  : 0.12009193003177643
[MASK]: 'زنجان'  : 0.0818941742181778
اراک مرکزی مریوان کردستان
شهر اراک در استان مرکزی  در نتیجه شهر مریوان در استان [MASK] می باشد.
[MASK]: 'فارس'  : 0.4900856614112854
[MASK]: 'خراس

In [7]:
Accuracy_currency = 0
y= df[df["cat"]=="currency"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "currency"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " واحد پول "+df["second"][i[0]] + " برای کشور " + df["first"][i[0]] + " است پس ارز و پول " + "[MASK]" + " برای کشور " + df["third"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_currency = Accuracy_currency + 1   
num = y["cat"].count()      
print(Accuracy_currency/num)     

سوئد کرون امارات درهم
 واحد پول کرون برای کشور سوئد است پس ارز و پول [MASK] برای کشور امارات است.
[MASK]: 'نیز'  : 0.24066373705863953
[MASK]: 'ان'  : 0.08819352090358734
[MASK]: 'دیگر'  : 0.06690553575754166
سوئیس فرانک تاجیکستان سامانی
 واحد پول فرانک برای کشور سوئیس است پس ارز و پول [MASK] برای کشور تاجیکستان است.
[MASK]: 'نیز'  : 0.1693984568119049
[MASK]: 'ان'  : 0.09727966785430908
[MASK]: 'دیگر'  : 0.06170953810214996
رومانی لئو بریتانیا پوند
 واحد پول لئو برای کشور رومانی است پس ارز و پول [MASK] برای کشور بریتانیا است.
[MASK]: 'نیز'  : 0.12562894821166992
[MASK]: 'ان'  : 0.1072237640619278
[MASK]: 'هم'  : 0.059644587337970734
ترکیه لیره افغانستان افغانی
 واحد پول لیره برای کشور ترکیه است پس ارز و پول [MASK] برای کشور افغانستان است.
[MASK]: 'نیز'  : 0.18416906893253326
[MASK]: 'دیگری'  : 0.07434811443090439
[MASK]: 'دیگر'  : 0.0616905577480793
چین یوان عراق دینار
 واحد پول یوان برای کشور چین است پس ارز و پول [MASK] برای کشور عراق است.
[MASK]: 'نیز'  : 0.16138052940368652
[MASK]:

In [8]:
Accuracy_capital = 0
y= df[df["cat"]=="capital"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "capital"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " شهر "+df["second"][i[0]] + " پایتخت کشور " + df["first"][i[0]] + " است پس شهر " + "[MASK]" + " پایتخت کشور " + df["third"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_capital = Accuracy_capital + 1   
num = y["cat"].count()      
print(Accuracy_capital/num)     

روسیه مسکو هلند آمستردام
 شهر مسکو پایتخت کشور روسیه است پس شهر [MASK] پایتخت کشور هلند است.
[MASK]: 'لندن'  : 0.10341990739107132
[MASK]: '##لند'  : 0.09529469162225723
[MASK]: 'عمان'  : 0.029793143272399902
کره‌جنوبی سئول آمریکا واشنگتن
 شهر سئول پایتخت کشور کره‌جنوبی است پس شهر [MASK] پایتخت کشور آمریکا است.
[MASK]: 'نیویورک'  : 0.8177030682563782
[MASK]: 'لندن'  : 0.01627461053431034
[MASK]: 'کنتاکی'  : 0.01066240668296814
ایران تهران اسپانیا مادرید
 شهر تهران پایتخت کشور ایران است پس شهر [MASK] پایتخت کشور اسپانیا است.
[MASK]: 'اسپانیا'  : 0.04977310448884964
[MASK]: 'دمشق'  : 0.04648655280470848
[MASK]: 'زنجان'  : 0.03746030107140541
ایران تهران افغانستان کابل
 شهر تهران پایتخت کشور ایران است پس شهر [MASK] پایتخت کشور افغانستان است.
[MASK]: 'مشهد'  : 0.2014915496110916
[MASK]: 'افغانستان'  : 0.13720181584358215
[MASK]: 'زنجان'  : 0.07877206802368164
بریتانیا لندن فنلاند هلسینکی
 شهر لندن پایتخت کشور بریتانیا است پس شهر [MASK] پایتخت کشور فنلاند است.
[MASK]: 'تهران'  : 0.116484992

In [9]:
Accuracy_family = 0
y= df[df["cat"]=="family"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "family"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت خانوادگی "+df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند نسبت خانوادگی " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_family = Accuracy_family + 1   
num = y["cat"].count()      
print(Accuracy_family/num)     

برادر خواهر پدر مادر
 نسبت خانوادگی برادر به خواهر مانند نسبت خانوادگی پدر به [MASK] است.
[MASK]: 'مادر'  : 0.6541892886161804
[MASK]: 'پسر'  : 0.08836635202169418
[MASK]: 'پدر'  : 0.0667133629322052
برادرها خواهرها شوهر عیال
 نسبت خانوادگی برادرها به خواهرها مانند نسبت خانوادگی شوهر به [MASK] است.
[MASK]: 'مادر'  : 0.1582661122083664
[MASK]: 'پسر'  : 0.08138175308704376
[MASK]: 'دختر'  : 0.07784987986087799
پدربزرگ مادربزرگ برادر خواهر
 نسبت خانوادگی پدربزرگ به مادربزرگ مانند نسبت خانوادگی برادر به [MASK] است.
[MASK]: 'مادر'  : 0.4135016202926636
[MASK]: 'برادر'  : 0.23254965245723724
[MASK]: 'پدر'  : 0.1999197006225586
بابابزرگ مامان‌بزرگ داماد عروس
 نسبت خانوادگی بابابزرگ به مامان‌بزرگ مانند نسبت خانوادگی داماد به [MASK] است.
[MASK]: 'پدر'  : 0.05418102443218231
[MASK]: 'او'  : 0.04109899327158928
[MASK]: 'مادر'  : 0.040195245295763016
داماد عروس برادرها خواهرها
 نسبت خانوادگی داماد به عروس مانند نسبت خانوادگی برادرها به [MASK] است.
[MASK]: 'مادر'  : 0.3785322606563568
[MASK]: 'پدر'

In [10]:
Accuracy = 0
y= df[df["cat"]=="gram_thirdperson"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_thirdperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " فعل "+df["first"][i[0]] + " سوم شخص مفرد و فعل " + df["second"][i[0]] + " سوم شخص جمع است در نتیجه فعل " + df["third"][i[0]] + " سوم شخص مفرد و فعل " + "[MASK]" + " سوم شخص جمع است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)     

برید بریدند رقصید رقصیدند
 فعل برید سوم شخص مفرد و فعل بریدند سوم شخص جمع است در نتیجه فعل رقصید سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: 'جمع'  : 0.19850026071071625
[MASK]: 'مراد'  : 0.021399134770035744
[MASK]: 'فعل'  : 0.017265262082219124
پرید پریدند آمد آمدند
 فعل پرید سوم شخص مفرد و فعل پریدند سوم شخص جمع است در نتیجه فعل آمد سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: 'امد'  : 0.9436482191085815
[MASK]: 'رفت'  : 0.0043782261200249195
[MASK]: 'بازگشت'  : 0.0031403549946844578
تازید تازیدند گفت گفتند
 فعل تازید سوم شخص مفرد و فعل تازیدند سوم شخص جمع است در نتیجه فعل گفت سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: 'گفت'  : 0.8718505501747131
[MASK]: 'کرد'  : 0.0045913285575807095
[MASK]: 'است'  : 0.0028545481618493795
شد شدند بوسید بوسیدند
 فعل شد سوم شخص مفرد و فعل شدند سوم شخص جمع است در نتیجه فعل بوسید سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: 'شد'  : 0.7441174387931824
[MASK]: 'شدند'  : 0.07719182223081589
[MASK]: 'شود'  : 0.017330162227153778


In [11]:
Accuracy = 0
y= df[df["cat"]=="gram_past"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_past"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "  مصدر "+df["third"][i[0]] + " فعل " + "[MASK]" + " است همانطور که مصدر " + df["first"][i[0]] + " فعل " + df["second"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)      

بریدن برید دمیدن دمید
  مصدر دمیدن فعل [MASK] است همانطور که مصدر بریدن فعل برید است.
[MASK]: '##ی'  : 0.6115239262580872
[MASK]: 'ان'  : 0.038365334272384644
[MASK]: '##ای'  : 0.015561294741928577
تازیدن تاخت دادن داد
  مصدر دادن فعل [MASK] است همانطور که مصدر تازیدن فعل تاخت است.
[MASK]: '##ی'  : 0.26172852516174316
[MASK]: 'ان'  : 0.034055110067129135
[MASK]: 'عبارت'  : 0.02728327177464962
شدن شد نگریستن نگریست
  مصدر نگریستن فعل [MASK] است همانطور که مصدر شدن فعل شد است.
[MASK]: '##ی'  : 0.3850245177745819
[MASK]: 'شده'  : 0.15304791927337646
[MASK]: '##ا'  : 0.020293574780225754
گفتن گفت رقصیدن رقصید
  مصدر رقصیدن فعل [MASK] است همانطور که مصدر گفتن فعل گفت است.
[MASK]: '##ی'  : 0.4691613018512726
[MASK]: 'گفت'  : 0.10708144307136536
[MASK]: 'ان'  : 0.032318271696567535
گفتن گفت خواستن خواست
  مصدر خواستن فعل [MASK] است همانطور که مصدر گفتن فعل گفت است.
[MASK]: 'گفت'  : 0.4414641857147217
[MASK]: '##ی'  : 0.1625087708234787
[MASK]: 'گفته'  : 0.02152896672487259
توانستن توانست رسید

In [12]:
Accuracy = 0
y= df[df["cat"]=="gram_firstperson"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_firstperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " من "+df["third"][i[0]] + "  پس ما " + "[MASK]" + " من " + df["first"][i[0]] + " و ما " + df["second"][i[0]] + " "
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

دادم دادیم یافتم یافتیم
 من یافتم  پس ما [MASK] من دادم و ما دادیم 
[MASK]: 'به'  : 0.27473634481430054
[MASK]: 'را'  : 0.14400145411491394
[MASK]: 'هم'  : 0.09625417739152908
زدم زدیم نامیدم نامیدیم
 من نامیدم  پس ما [MASK] من زدم و ما زدیم 
[MASK]: 'هم'  : 0.08082239329814911
[MASK]: 'به'  : 0.06937608122825623
[MASK]: 'را'  : 0.06537292897701263
نوشتم نوشتیم گفتم گفتیم
 من گفتم  پس ما [MASK] من نوشتم و ما نوشتیم 
[MASK]: 'به'  : 0.17944695055484772
[MASK]: 'را'  : 0.11824307590723038
[MASK]: 'از'  : 0.07933424413204193
دانستم دانستیم بلعیدم بلعیدیم
 من بلعیدم  پس ما [MASK] من دانستم و ما دانستیم 
[MASK]: 'را'  : 0.3862951993942261
[MASK]: 'هم'  : 0.08532244712114334
[MASK]: 'به'  : 0.04412073269486427
گردیدم گردیدیم یافتم یافتیم
 من یافتم  پس ما [MASK] من گردیدم و ما گردیدیم 
[MASK]: 'به'  : 0.07574494928121567
[MASK]: 'از'  : 0.05970778688788414
[MASK]: '##شین'  : 0.03043581359088421
گردیدم گردیدیم بوسیدم بوسیدیم
 من بوسیدم  پس ما [MASK] من گردیدم و ما گردیدیم 
[MASK]: 'به'  : 0.06

In [13]:
Accuracy = 0
y= df[df["cat"]=="gram_adj2adv"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_adj2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "  "+ df["first"][i[0]] + " صفت و " + df["second"][i[0]] + " قید است. در نتیجه " + df["third"][i[0]] + " صفت و " + "[MASK]" + " قید است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

مجدد مجددا مخصوص مخصوصا
  مجدد صفت و مجددا قید است. در نتیجه مخصوص صفت و [MASK] قید است.
[MASK]: 'نه'  : 0.14233966171741486
[MASK]: 'سپس'  : 0.04236554726958275
[MASK]: 'دوباره'  : 0.026660438627004623
عمده عمدتا صمیمی صمیمانه
  عمده صفت و عمدتا قید است. در نتیجه صمیمی صفت و [MASK] قید است.
[MASK]: 'نه'  : 0.2030404806137085
[MASK]: 'غیر'  : 0.08287172019481659
[MASK]: 'نیز'  : 0.038336437195539474
دقیق دقیقا عادل عادلانه
  دقیق صفت و دقیقا قید است. در نتیجه عادل صفت و [MASK] قید است.
[MASK]: 'غیر'  : 0.09689155220985413
[MASK]: 'نه'  : 0.05801532790064812
[MASK]: 'عاد'  : 0.048534587025642395
غالب غالبا جدا جداگانه
  غالب صفت و غالبا قید است. در نتیجه جدا صفت و [MASK] قید است.
[MASK]: 'غالبا'  : 0.33540821075439453
[MASK]: 'گاهی'  : 0.08050327003002167
[MASK]: 'معمولا'  : 0.06783808022737503
متناظر متناظرا قبل قبلا
  متناظر صفت و متناظرا قید است. در نتیجه قبل صفت و [MASK] قید است.
[MASK]: 'بعد'  : 0.4012751877307892
[MASK]: 'قبل'  : 0.09580715000629425
[MASK]: 'اخر'  : 0.044692773371

In [14]:
Accuracy = 0
y= df[df["cat"]=="gram_noun2adv"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_noun2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت " + df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند نسبت " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

اتفاق اتفاقا شب شبانه
 نسبت اتفاق به اتفاقا مانند نسبت شب به [MASK] است.
[MASK]: 'روز'  : 0.4592703580856323
[MASK]: 'شب'  : 0.09859777241945267
[MASK]: 'زمان'  : 0.07105043530464172
حتم حتما اساس اساسا
 نسبت حتم به حتما مانند نسبت اساس به [MASK] است.
[MASK]: 'اساس'  : 0.3589920103549957
[MASK]: 'صفر'  : 0.18576118350028992
[MASK]: 'جرم'  : 0.0452507808804512
اصل اصلا انسان انسانوار
 نسبت اصل به اصلا مانند نسبت انسان به [MASK] است.
[MASK]: 'اصل'  : 0.5585536360740662
[MASK]: 'انسان'  : 0.021273121237754822
[MASK]: 'ماده'  : 0.018479079008102417
عمل عملا اجبار اجبارا
 نسبت عمل به عملا مانند نسبت اجبار به [MASK] است.
[MASK]: 'عمل'  : 0.22234301269054413
[MASK]: 'فرد'  : 0.03530729189515114
[MASK]: 'حرکت'  : 0.02793191373348236
اجبار اجبارا کودک کودکانه
 نسبت اجبار به اجبارا مانند نسبت کودک به [MASK] است.
[MASK]: 'مادر'  : 0.2190786600112915
[MASK]: 'کودکان'  : 0.11339234560728073
[MASK]: 'خانواده'  : 0.07975120842456818
دائم دائما اتفاق اتفاقا
 نسبت دائم به دائما مانند نسبت اتفاق به [MAS

In [15]:
Accuracy = 0
y= df[df["cat"]=="gram_antonym"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_antonym"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

خشنود ناخشنود موفق ناموفق
خشنود به ناخشنود مانند موفق به [MASK] است.
[MASK]: 'کار'  : 0.13138672709465027
[MASK]: 'زندگی'  : 0.059533100575208664
[MASK]: 'مرگ'  : 0.027515318244695663
درست نادرست پخته ناپخته
درست به نادرست مانند پخته به [MASK] است.
[MASK]: '##تر'  : 0.1893381029367447
[MASK]: 'رنگ'  : 0.030528463423252106
[MASK]: 'کار'  : 0.024230236187577248
مناسب نامناسب همزمان ناهمزمان
مناسب به نامناسب مانند همزمان به [MASK] است.
[MASK]: '##تر'  : 0.09568536281585693
[MASK]: 'همراه'  : 0.09260991960763931
[MASK]: 'کار'  : 0.04620065912604332
مربوط نامربوط منسجم نامنسجم
مربوط به نامربوط مانند منسجم به [MASK] است.
[MASK]: 'فارسی'  : 0.06747344881296158
[MASK]: 'ان'  : 0.04987502843141556
[MASK]: 'شکل'  : 0.022115981206297874
بینا نابینا درست نادرست
بینا به نابینا مانند درست به [MASK] است.
[MASK]: 'معنی'  : 0.021974191069602966
[MASK]: 'کار'  : 0.0216278824955225
[MASK]: 'انسان'  : 0.020318521186709404
کارآمد ناکارآمد هنجار ناهنجار
کارآمد به ناکارآمد مانند هنجار به [MASK] است.
[MASK]: 

In [16]:
Accuracy = 0
y= df[df["cat"]=="gram_comparative"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_comparative"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت " + df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند نسبت " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

سریع سریعتر بلند بلندتر
 نسبت سریع به سریعتر مانند نسبت بلند به [MASK] است.
[MASK]: 'متوسط'  : 0.09361278265714645
[MASK]: 'سرعت'  : 0.08244204521179199
[MASK]: 'بلند'  : 0.07534634321928024
بزرگ بزرگتر شدید شدیدتر
 نسبت بزرگ به بزرگتر مانند نسبت شدید به [MASK] است.
[MASK]: 'کوچک'  : 0.1658000946044922
[MASK]: 'بزرگ'  : 0.145115464925766
[MASK]: 'متوسط'  : 0.1021316722035408
مهم مهمتر بارز بارزتر
 نسبت مهم به مهمتر مانند نسبت بارز به [MASK] است.
[MASK]: 'بزرگ'  : 0.0924774557352066
[MASK]: 'مهم'  : 0.0789758637547493
[MASK]: 'کوچک'  : 0.05768731236457825
به بهتر سرد سردتر
 نسبت به به بهتر مانند نسبت سرد به [MASK] است.
[MASK]: '##تر'  : 0.21781861782073975
[MASK]: 'گرم'  : 0.07535195350646973
[MASK]: 'صفر'  : 0.04434264451265335
واضح واضحتر بیش بیشتر
 نسبت واضح به واضحتر مانند نسبت بیش به [MASK] است.
[MASK]: 'بیش'  : 0.3239390254020691
[MASK]: 'کم'  : 0.24127669632434845
[MASK]: 'صفر'  : 0.057683397084474564
کم کمتر بد بدتر
 نسبت کم به کمتر مانند نسبت بد به [MASK] است.
[MASK]: 'کمتر'  :

In [17]:
Accuracy = 0
y= df[df["cat"]=="gram_nationality"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_nationality"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " مردم " + df["second"][i[0]] + " اهل کشور " + df["first"][i[0]] + " هستند. و مردم " + "[MASK]" + " اهل کشور " + df["third"][i[0]] + " هستند."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

اتریش اتریشی اسپانیا اسپانیایی
 مردم اتریشی اهل کشور اتریش هستند. و مردم [MASK] اهل کشور اسپانیا هستند.
[MASK]: 'اسپانیا'  : 0.48986729979515076
[MASK]: 'اسپانیایی'  : 0.17216339707374573
[MASK]: 'دیگر'  : 0.062355827540159225
سوئد سوئدی فنلاند فنلاندی
 مردم سوئدی اهل کشور سوئد هستند. و مردم [MASK] اهل کشور فنلاند هستند.
[MASK]: 'دیگر'  : 0.27652353048324585
[MASK]: 'انگلیسی'  : 0.07977279275655746
[MASK]: 'ان'  : 0.07308009266853333
بحرین بحرینی عراق عراقی
 مردم بحرینی اهل کشور بحرین هستند. و مردم [MASK] اهل کشور عراق هستند.
[MASK]: 'ان'  : 0.19337008893489838
[MASK]: 'دیگر'  : 0.0857117623090744
[MASK]: '##ش'  : 0.06468784064054489
کویت کویتی سوریه سوری
 مردم کویتی اهل کشور کویت هستند. و مردم [MASK] اهل کشور سوریه هستند.
[MASK]: 'سوریه'  : 0.3744276463985443
[MASK]: 'دیگر'  : 0.0781632587313652
[MASK]: 'ان'  : 0.07008420675992966
مصر مصری کرواسی کروات
 مردم مصری اهل کشور مصر هستند. و مردم [MASK] اهل کشور کرواسی هستند.
[MASK]: 'مصر'  : 0.12852302193641663
[MASK]: 'دیگر'  : 0.099453367

In [18]:
Accuracy = 0
y= df[df["cat"]=="gram_plural"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_plural"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " " + df["first"][i[0]] + " جمع " + df["second"][i[0]] + " است و " + df["third"][i[0]] + " جمع " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

تالیف تالیفات مخلوق مخلوقات
 تالیف جمع تالیفات است و مخلوق جمع [MASK] است.
[MASK]: 'ان'  : 0.3297886252403259
[MASK]: 'انها'  : 0.0792476087808609
[MASK]: '##ی'  : 0.07641895115375519
اثر اثرات ارزش ارزشها
 اثر جمع اثرات است و ارزش جمع [MASK] است.
[MASK]: 'ان'  : 0.3189881443977356
[MASK]: '##ی'  : 0.11591137200593948
[MASK]: 'انها'  : 0.1027236133813858
انقلابی انقلابیون مورد موارد
 انقلابی جمع انقلابیون است و مورد جمع [MASK] است.
[MASK]: 'مردم'  : 0.24239052832126617
[MASK]: 'انان'  : 0.10054605454206467
[MASK]: '##ی'  : 0.07323906570672989
حواری حواریون مورد موارد
 حواری جمع حواریون است و مورد جمع [MASK] است.
[MASK]: '##ی'  : 0.1571516990661621
[MASK]: 'ان'  : 0.12830837070941925
[MASK]: 'انها'  : 0.045120831578969955
مسئول مسئولین آزاده آزادگان
 مسئول جمع مسئولین است و آزاده جمع [MASK] است.
[MASK]: 'مردم'  : 0.17526523768901825
[MASK]: '##ها'  : 0.07312743365764618
[MASK]: '##ی'  : 0.07186919450759888
مسافر مسافرین طلا طلاها
 مسافر جمع مسافرین است و طلا جمع [MASK] است.
[MASK]: 'ان'

In [19]:
Accuracy= 0
y= df[df["cat"]=="city"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "city"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " "+df["first"][i[0]] + " در " + "[MASK]" + df["second"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = " "+df["third"][i[0]] + " در " + predicted + "[MASK]" + " است."
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)      


اهواز خوزستان قشم هرمزگان
 اهواز در [MASK]خوزستان است.
[MASK]: 'استان'  : 0.8101209402084351
[MASK]: 'کشور'  : 0.09197814762592316
[MASK]: 'جنوب'  : 0.024863000959157944
 قشم در استان[MASK] است.
[MASK]: 'اصفهان'  : 0.23169325292110443
[MASK]: 'کرمان'  : 0.17971815168857574
[MASK]: 'فارس'  : 0.17607173323631287
شیراز فارس دامغان سمنان
 شیراز در [MASK]فارس است.
[MASK]: 'استان'  : 0.9595129489898682
[MASK]: 'کشور'  : 0.016029924154281616
[MASK]: 'ایالت'  : 0.006484882906079292
 دامغان در استان[MASK] است.
[MASK]: 'ایران'  : 0.4069327116012573
[MASK]: 'اذربایجان'  : 0.20130763947963715
[MASK]: 'مرکزی'  : 0.060159195214509964
رشت گیلان اسلام‌شهر تهران
 رشت در [MASK]گیلان است.
[MASK]: 'استان'  : 0.8055057525634766
[MASK]: 'کشور'  : 0.04286566376686096
[MASK]: 'شمال'  : 0.036363452672958374
 اسلام‌شهر در استان[MASK] است.
[MASK]: 'کرمان'  : 0.21667072176933289
[MASK]: 'تهران'  : 0.2147563099861145
[MASK]: 'فارس'  : 0.18646858632564545
بندرعباس هرمزگان ملارد تهران
 بندرعباس در [MASK]هرمزگان است.

In [20]:
Accuracy= 0
y= df[df["cat"]=="capital"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "capital"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = " "+"[MASK]" + " " + predicted + " " + df["third"][i[0]] + " است."
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)   

روسیه مسکو هلند آمستردام
مسکو [MASK] روسیه است.
[MASK]: 'در'  : 0.4795851409435272
[MASK]: 'کشور'  : 0.31101787090301514
[MASK]: 'استان'  : 0.049701910465955734
 [MASK] در هلند است.
[MASK]: 'شهر'  : 0.15052641928195953
[MASK]: 'شهری'  : 0.09396160393953323
[MASK]: 'روستایی'  : 0.02368791587650776
کره‌جنوبی سئول آمریکا واشنگتن
سئول [MASK] کره‌جنوبی است.
[MASK]: 'یک'  : 0.2912152409553528
[MASK]: 'کشور'  : 0.16920791566371918
[MASK]: 'نام'  : 0.07841600477695465
 [MASK] یک آمریکا است.
[MASK]: 'امریکا'  : 0.01472187414765358
[MASK]: 'ایران'  : 0.011027383618056774
[MASK]: 'ایالات'  : 0.008416227996349335
ایران تهران اسپانیا مادرید
تهران [MASK] ایران است.
[MASK]: 'در'  : 0.444717675447464
[MASK]: '،'  : 0.2873581349849701
[MASK]: 'کشور'  : 0.07938764989376068
 [MASK] در اسپانیا است.
[MASK]: 'روستایی'  : 0.07293089479207993
[MASK]: 'شهر'  : 0.0668710470199585
[MASK]: 'دهستان'  : 0.05150184780359268
ایران تهران افغانستان کابل
تهران [MASK] ایران است.
[MASK]: 'در'  : 0.444717675447464
[MASK]: 

In [21]:
Accuracy= 0
y= df[df["cat"]=="currency"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "currency"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است."
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)   

سوئد کرون امارات درهم
کرون [MASK] سوئد است.
[MASK]: 'کشور'  : 0.21703748404979706
[MASK]: 'در'  : 0.11467083543539047
[MASK]: 'شهر'  : 0.1058410108089447
کشورامارات [MASK] است.
[MASK]: '##ی'  : 0.1760983169078827
[MASK]: 'کشور'  : 0.0344209298491478
[MASK]: 'واقع'  : 0.03012784570455551
سوئیس فرانک تاجیکستان سامانی
فرانک [MASK] سوئیس است.
[MASK]: 'اهل'  : 0.3590242266654968
[MASK]: 'متولد'  : 0.19289489090442657
[MASK]: 'زاده'  : 0.0864642858505249
اهلتاجیکستان [MASK] است.
[MASK]: 'ایران'  : 0.26188355684280396
[MASK]: '##ی'  : 0.10717086493968964
[MASK]: 'ایرانی'  : 0.06236531212925911
رومانی لئو بریتانیا پوند
لئو [MASK] رومانی است.
[MASK]: 'بازیگر'  : 0.3006412982940674
[MASK]: 'خواننده'  : 0.22569364309310913
[MASK]: 'هنرپیشه'  : 0.10792484134435654
بازیگربریتانیا [MASK] است.
[MASK]: '##ی'  : 0.14893338084220886
[MASK]: '##یی'  : 0.11004052311182022
[MASK]: '##زی'  : 0.025097347795963287
ترکیه لیره افغانستان افغانی
لیره [MASK] ترکیه است.
[MASK]: 'در'  : 0.31304487586021423
[MASK]: '

In [22]:
Accuracy= 0
y= df[df["cat"]=="family"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "family"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

برادر خواهر پدر مادر
خواهر [MASK] برادر است.
[MASK]: 'یک'  : 0.2774149477481842
[MASK]: 'و'  : 0.11078405380249023
[MASK]: '##ش'  : 0.07445964962244034
یکپدر [MASK] است. 
[MASK]: '##ی'  : 0.0559479258954525
[MASK]: '##ه'  : 0.047091227024793625
[MASK]: '##ان'  : 0.04649162292480469
برادرها خواهرها شوهر عیال
خواهرها [MASK] برادرها است.
[MASK]: 'و'  : 0.15499809384346008
[MASK]: 'هم'  : 0.11205566674470901
[MASK]: 'همه'  : 0.09839010238647461
وشوهر [MASK] است. 
[MASK]: 'ایرانی'  : 0.0409611314535141
[MASK]: 'امریکایی'  : 0.03391421213746071
[MASK]: 'مشهور'  : 0.024887198582291603
پدربزرگ مادربزرگ برادر خواهر
مادربزرگ [MASK] پدربزرگ است.
[MASK]: 'از'  : 0.09127308428287506
[MASK]: 'و'  : 0.08761948347091675
[MASK]: 'پسر'  : 0.06908576935529709
ازبرادر [MASK] است. 
[MASK]: '##ی'  : 0.21269729733467102
[MASK]: '##ان'  : 0.052154093980789185
[MASK]: '##ه'  : 0.04289988800883293
بابابزرگ مامان‌بزرگ داماد عروس
مامان‌بزرگ [MASK] بابابزرگ است.
[MASK]: 'پسر'  : 0.3114369809627533
[MASK]: 'نام'  :

In [23]:
Accuracy= 0
y= df[df["cat"]=="gram_thirdperson"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_thirdperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت " + df["second"][i[0]] + " به " +df["first"][i[0]]  + " " + "[MASK]"+ " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

برید بریدند رقصید رقصیدند
 نسبت بریدند به برید [MASK] است.
[MASK]: '##ن'  : 0.4136664569377899
[MASK]: '##ه'  : 0.0795106515288353
[MASK]: '##گی'  : 0.039073582738637924
##نرقصید [MASK] است. 
[MASK]: '##ی'  : 0.2637655436992645
[MASK]: '##ه'  : 0.07778816670179367
[MASK]: 'شده'  : 0.022046266123652458
پرید پریدند آمد آمدند
 نسبت پریدند به پرید [MASK] است.
[MASK]: '##ن'  : 0.4453730285167694
[MASK]: '##نده'  : 0.03927445039153099
[MASK]: '##ند'  : 0.0386078916490078
##نآمد [MASK] است. 
[MASK]: '##ست'  : 0.061706747859716415
[MASK]: '##عی'  : 0.04143601283431053
[MASK]: '##بیر'  : 0.04136434942483902
تازید تازیدند گفت گفتند
 نسبت تازیدند به تازید [MASK] است.
[MASK]: '##ن'  : 0.39660367369651794
[MASK]: '##ند'  : 0.22666604816913605
[MASK]: '##ان'  : 0.048721082508563995
##نگفت [MASK] است. 
[MASK]: '##ی'  : 0.2454553097486496
[MASK]: '##ن'  : 0.04376988857984543
[MASK]: '##ا'  : 0.03058651089668274
شد شدند بوسید بوسیدند
 نسبت شدند به شد [MASK] است.
[MASK]: '##ی'  : 0.19075807929039001
[MA

In [24]:
Accuracy= 0
y= df[df["cat"]=="gram_past"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_past"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

بریدن برید دمیدن دمید
برید [MASK] بریدن است.
[MASK]: '##ن'  : 0.5509887933731079
[MASK]: '##ه'  : 0.03847062587738037
[MASK]: '##ر'  : 0.03307155892252922
##ندمیدن [MASK] است. 
[MASK]: 'ان'  : 0.052739571779966354
[MASK]: 'شده'  : 0.021655384451150894
[MASK]: 'عبارت'  : 0.020007701590657234
تازیدن تاخت دادن داد
تاخت [MASK] تازیدن است.
[MASK]: '##ن'  : 0.28429779410362244
[MASK]: '##ر'  : 0.0829700455069542
[MASK]: 'و'  : 0.047156259417533875
##ندادن [MASK] است. 
[MASK]: 'عبارت'  : 0.032647814601659775
[MASK]: 'ان'  : 0.031233230605721474
[MASK]: 'لازم'  : 0.02687162719666958
شدن شد نگریستن نگریست
شد [MASK] شدن است.
[MASK]: '##ی'  : 0.22446559369564056
[MASK]: '##ه'  : 0.1761407107114792
[MASK]: '##ار'  : 0.12402896583080292
##ینگریستن [MASK] است. 
[MASK]: '##نده'  : 0.060127805918455124
[MASK]: 'کسی'  : 0.020216690376400948
[MASK]: 'عبارت'  : 0.01950058341026306
گفتن گفت رقصیدن رقصید
گفت [MASK] گفتن است.
[MASK]: '##ن'  : 0.8277590870857239
[MASK]: '##نی'  : 0.058645736426115036
[MASK]:

In [25]:
Accuracy= 0
y= df[df["cat"]=="gram_firstperson"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_firstperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

دادم دادیم یافتم یافتیم
دادیم [MASK] دادم است.
[MASK]: 'و'  : 0.07341153174638748
[MASK]: '،'  : 0.05528134107589722
[MASK]: 'به'  : 0.02743873931467533
ویافتم [MASK] است. 
[MASK]: '##د'  : 0.06428436934947968
[MASK]: '##ن'  : 0.0501580536365509
[MASK]: '##ر'  : 0.04390314966440201
زدم زدیم نامیدم نامیدیم
زدیم [MASK] زدم است.
[MASK]: 'و'  : 0.10296232253313065
[MASK]: '،'  : 0.06191697716712952
[MASK]: 'یک'  : 0.05268308147788048
ونامیدم [MASK] است. 
[MASK]: '##ی'  : 0.08229043334722519
[MASK]: '##ان'  : 0.07842815667390823
[MASK]: '##ا'  : 0.04496690258383751
نوشتم نوشتیم گفتم گفتیم
نوشتیم [MASK] نوشتم است.
[MASK]: 'و'  : 0.08682495355606079
[MASK]: 'یک'  : 0.08317939937114716
[MASK]: '،'  : 0.07155920565128326
وگفتم [MASK] است. 
[MASK]: 'عشق'  : 0.035415519028902054
[MASK]: 'ان'  : 0.02732391096651554
[MASK]: 'این'  : 0.01806379109621048
دانستم دانستیم بلعیدم بلعیدیم
دانستیم [MASK] دانستم است.
[MASK]: '،'  : 0.49638888239860535
[MASK]: 'و'  : 0.06664469838142395
[MASK]: '.'  : 0.0651

In [26]:
Accuracy= 0
y= df[df["cat"]=="gram_adj2adv"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_adj2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

مجدد مجددا مخصوص مخصوصا
مجددا [MASK] مجدد است.
[MASK]: 'و'  : 0.27786532044410706
[MASK]: '،'  : 0.06046896427869797
[MASK]: 'هم'  : 0.01712346263229847
ومخصوص [MASK] است. 
[MASK]: 'ان'  : 0.11347765475511551
[MASK]: '##ا'  : 0.06865421682596207
[MASK]: 'ایران'  : 0.031763844192028046
عمده عمدتا صمیمی صمیمانه
عمدتا [MASK] عمده است.
[MASK]: 'و'  : 0.07413601130247116
[MASK]: 'در'  : 0.06687654554843903
[MASK]: '،'  : 0.05412502586841583
وصمیمی [MASK] است. 
[MASK]: 'زاده'  : 0.17764820158481598
[MASK]: 'معروف'  : 0.044563643634319305
[MASK]: '##ه'  : 0.03363923355937004
دقیق دقیقا عادل عادلانه
دقیقا [MASK] دقیق است.
[MASK]: ':'  : 0.14647096395492554
[MASK]: '،'  : 0.13678738474845886
[MASK]: 'و'  : 0.10353029519319534
:عادل [MASK] است. 
[MASK]: '،'  : 0.02579355053603649
[MASK]: 'خان'  : 0.0256927702575922
[MASK]: 'زاده'  : 0.024741247296333313
غالب غالبا جدا جداگانه
غالبا [MASK] غالب است.
[MASK]: 'بسیار'  : 0.09609111398458481
[MASK]: '،'  : 0.05781613290309906
[MASK]: 'ما'  : 0.053741

In [27]:
Accuracy= 0
y= df[df["cat"]=="gram_noun2adv"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_noun2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

اتفاق اتفاقا شب شبانه
اتفاقا [MASK] اتفاق است.
[MASK]: 'یک'  : 0.261593759059906
[MASK]: 'این'  : 0.14018285274505615
[MASK]: 'همین'  : 0.06383007764816284
یکشب [MASK] است. 
[MASK]: '##خ'  : 0.18450094759464264
[MASK]: '##ک'  : 0.09608732163906097
[MASK]: '##خته'  : 0.039686255156993866
حتم حتما اساس اساسا
حتما [MASK] حتم است.
[MASK]: 'یک'  : 0.07112009078264236
[MASK]: ':'  : 0.056765127927064896
[MASK]: '،'  : 0.03887493535876274
یکاساس [MASK] است. 
[MASK]: '##ونی'  : 0.08492748439311981
[MASK]: '##پی'  : 0.08325354754924774
[MASK]: '##ون'  : 0.03339621052145958
اصل اصلا انسان انسانوار
اصلا [MASK] اصل است.
[MASK]: 'همان'  : 0.07317017763853073
[MASK]: 'یک'  : 0.07093438506126404
[MASK]: 'در'  : 0.06257723271846771
همانانسان [MASK] است. 
[MASK]: '##تر'  : 0.030624739825725555
[MASK]: '##کی'  : 0.014140567742288113
[MASK]: 'بزرگ'  : 0.010560065507888794
عمل عملا اجبار اجبارا
عملا [MASK] عمل است.
[MASK]: 'یک'  : 0.06027417629957199
[MASK]: 'همان'  : 0.04881002753973007
[MASK]: '،'  : 0.

In [28]:
Accuracy= 0
y= df[df["cat"]=="gram_antonym"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_antonym"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["first"][i[0]] + " و " + df["second"][i[0]]+"[MASK]" + " یکدیگر هستند."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

خشنود ناخشنود موفق ناموفق
خشنود و ناخشنود[MASK] یکدیگر هستند.
[MASK]: 'هم'  : 0.09552275389432907
[MASK]: '،'  : 0.06714913249015808
[MASK]: 'با'  : 0.055401816964149475
همموفق [MASK] است. 
[MASK]: '##ی'  : 0.09956672042608261
[MASK]: '##ان'  : 0.07335249334573746
[MASK]: '##ین'  : 0.03936736658215523
درست نادرست پخته ناپخته
درست و نادرست[MASK] یکدیگر هستند.
[MASK]: 'با'  : 0.2763713598251343
[MASK]: 'از'  : 0.15565428137779236
[MASK]: 'بودن'  : 0.09979806840419769
باپخته [MASK] است. 
[MASK]: '##ای'  : 0.28604501485824585
[MASK]: 'شده'  : 0.059193335473537445
[MASK]: '##ا'  : 0.057167280465364456
مناسب نامناسب همزمان ناهمزمان
مناسب و نامناسب[MASK] یکدیگر هستند.
[MASK]: 'به'  : 0.4589375853538513
[MASK]: 'با'  : 0.18008558452129364
[MASK]: 'از'  : 0.13712134957313538
بههمزمان [MASK] است. 
[MASK]: 'امده'  : 0.02803773246705532
[MASK]: 'معروف'  : 0.02712376043200493
[MASK]: 'مشهور'  : 0.024493969976902008
مربوط نامربوط منسجم نامنسجم
مربوط و نامربوط[MASK] یکدیگر هستند.
[MASK]: 'به'  : 0.43

In [29]:
Accuracy= 0
y= df[df["cat"]=="gram_comparative"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_comparative"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ "[MASK]" + " "   +  df["third"][i[0]] + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10) 

سریع سریعتر بلند بلندتر
سریعتر [MASK] سریع است.
[MASK]: 'و'  : 0.5347017049789429
[MASK]: '،'  : 0.12130160629749298
[MASK]: ':'  : 0.03923444449901581
و[MASK] بلند است. 
[MASK]: 'بسیار'  : 0.23587554693222046
[MASK]: 'نیز'  : 0.041191305965185165
[MASK]: 'گاهی'  : 0.02249571681022644
بزرگ بزرگتر شدید شدیدتر
بزرگتر [MASK] بزرگ است.
[MASK]: 'از'  : 0.6348297595977783
[MASK]: 'و'  : 0.17112857103347778
[MASK]: '،'  : 0.014418519102036953
از[MASK] شدید است. 
[MASK]: 'نظر'  : 0.08666545152664185
[MASK]: 'نوع'  : 0.06655316054821014
[MASK]: '##مای'  : 0.04676828905940056
مهم مهمتر بارز بارزتر
مهمتر [MASK] مهم است.
[MASK]: 'و'  : 0.5010991096496582
[MASK]: '،'  : 0.14229729771614075
[MASK]: 'یا'  : 0.026045501232147217
و[MASK] بارز است. 
[MASK]: 'بسیار'  : 0.0748867392539978
[MASK]: 'نیز'  : 0.02115141972899437
[MASK]: '##ادی'  : 0.016358334571123123
به بهتر سرد سردتر
بهتر [MASK] به است.
[MASK]: ':'  : 0.051724936813116074
[MASK]: 'معنی'  : 0.040443554520606995
[MASK]: '،'  : 0.0344945713877

In [30]:
Accuracy= 0
y= df[df["cat"]=="gram_nationality"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_nationality"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+" "+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10) 

اتریش اتریشی اسپانیا اسپانیایی
اتریشی [MASK] اتریش است.
[MASK]: 'در'  : 0.28926604986190796
[MASK]: 'کشور'  : 0.19034138321876526
[MASK]: 'یک'  : 0.0779063031077385
در اسپانیا [MASK] است. 
[MASK]: 'واقع'  : 0.6985306739807129
[MASK]: 'موجود'  : 0.040645286440849304
[MASK]: 'امده'  : 0.02721373178064823
سوئد سوئدی فنلاند فنلاندی
سوئدی [MASK] سوئد است.
[MASK]: 'کشور'  : 0.39408445358276367
[MASK]: 'شهر'  : 0.060148295015096664
[MASK]: 'ایالت'  : 0.05225590243935585
کشور فنلاند [MASK] است. 
[MASK]: 'واقع'  : 0.08923282474279404
[MASK]: '##ی'  : 0.06894484162330627
[MASK]: 'عضو'  : 0.03360282629728317
بحرین بحرینی عراق عراقی
بحرینی [MASK] بحرین است.
[MASK]: 'کشور'  : 0.07472815364599228
[MASK]: 'در'  : 0.053378812968730927
[MASK]: 'مرکز'  : 0.04102696105837822
کشور عراق [MASK] است. 
[MASK]: 'عراق'  : 0.08109019696712494
[MASK]: 'ایران'  : 0.07318089157342911
[MASK]: '##ی'  : 0.0612829253077507
کویت کویتی سوریه سوری
کویتی [MASK] کویت است.
[MASK]: 'استان'  : 0.09057620167732239
[MASK]: 'کشور

In [31]:
Accuracy= 0
y= df[df["cat"]=="gram_plural"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_plural"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+ df["third"][i[0]] + " " + predicted  +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10) 

تالیف تالیفات مخلوق مخلوقات
تالیفات [MASK] تالیف است.
[MASK]: 'و'  : 0.17734098434448242
[MASK]: 'چند'  : 0.12865720689296722
[MASK]: 'سه'  : 0.04323117807507515
مخلوق و[MASK] است. 
[MASK]: '##طن'  : 0.05652615427970886
[MASK]: 'شاعر'  : 0.016358530148863792
[MASK]: '##ام'  : 0.013466672971844673
اثر اثرات ارزش ارزشها
اثرات [MASK] اثر است.
[MASK]: 'یک'  : 0.32471102476119995
[MASK]: 'در'  : 0.03566258028149605
[MASK]: '،'  : 0.034342776983976364
ارزش یک[MASK] است. 
[MASK]: 'عبارت'  : 0.04226812720298767
[MASK]: 'عدد'  : 0.03175648674368858
[MASK]: 'رقم'  : 0.02685455046594143
انقلابی انقلابیون مورد موارد
انقلابیون [MASK] انقلابی است.
[MASK]: 'و'  : 0.1070709079504013
[MASK]: '##ی'  : 0.10460906475782394
[MASK]: '،'  : 0.09997863322496414
مورد و[MASK] است. 
[MASK]: '##طن'  : 0.1365734189748764
[MASK]: '##قف'  : 0.04111434519290924
[MASK]: '##ام'  : 0.02628764510154724
حواری حواریون مورد موارد
حواریون [MASK] حواری است.
[MASK]: 'یک'  : 0.24508796632289886
[MASK]: 'نوعی'  : 0.0714716091752